In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [7]:
model = Sequential([tf.keras.Input([16,]), layers.Dense(50), layers.Dense(20)])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                850       
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
Total params: 1,870
Trainable params: 1,870
Non-trainable params: 0
_________________________________________________________________


2022-07-28 13:20:39.507254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-28 13:20:40.561117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6476 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:03:00.0, compute capability: 7.5
2022-07-28 13:20:40.561702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6647 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:04:00.0, compute capability: 7.5


In [11]:
(x_trainval, y_trainval), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(type(x_trainval), x_trainval.shape)
print(type(x_trainval), x_trainval.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_trainval, y_trainval, test_size=1/6, shuffle=True, stratify=y_trainval, random_state=34)

print(type(x_train), x_train.shape)
print(type(x_train), x_train.shape)

<class 'numpy.ndarray'> (60000, 28, 28)
<class 'numpy.ndarray'> (60000, 28, 28)
<class 'numpy.ndarray'> (50000, 28, 28)
<class 'numpy.ndarray'> (50000, 28, 28)


In [12]:
x_train, x_valid, x_test = x_train.astype('float32') / 255, x_valid.astype('float32') / 255, x_test.astype('float32') / 255

print(x_train.shape)

x_train=x_train.reshape(-1,28,28,1)
x_valid=x_valid.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)

print(x_train.shape)

(50000, 28, 28)
(50000, 28, 28, 1)


In [15]:
cnn = Sequential(
    [
        layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPool2D(3, strides=2, padding='same'),
        layers.Conv2D(64, 3, strides=1, padding='same', activation='relu'),
        layers.MaxPool2D(3, strides=2, padding='same'),
        ## feature extraction
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax'),
    ]
)

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [19]:
cnn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    metrics=['accuracy']
)

In [21]:
result=cnn.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_valid, y_valid))

Epoch 1/5


2022-07-28 13:50:13.772822: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


391/391 [==============================] - 5s 7ms/step - loss: 0.4222 - accuracy: 0.8659 - val_loss: 0.1350 - val_accuracy: 0.9584
Epoch 2/5
391/391 [==============================] - 2s 6ms/step - loss: 0.0921 - accuracy: 0.9710 - val_loss: 0.0984 - val_accuracy: 0.9688
Epoch 3/5
391/391 [==============================] - 2s 6ms/step - loss: 0.0612 - accuracy: 0.9814 - val_loss: 0.0543 - val_accuracy: 0.9827
Epoch 4/5
391/391 [==============================] - 2s 6ms/step - loss: 0.0476 - accuracy: 0.9847 - val_loss: 0.0469 - val_accuracy: 0.9866
Epoch 5/5
391/391 [==============================] - 2s 6ms/step - loss: 0.0400 - accuracy: 0.9875 - val_loss: 0.0473 - val_accuracy: 0.9844


In [23]:
test_result = cnn.evaluate(x_test, y_test)
print('test_loss : ', test_result[0])
print('test_acc : ', test_result[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.0381 - accuracy: 0.9865
test_loss :  0.03814633935689926
test_acc :  0.9865000247955322
